In [1]:
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.empty_cache()
print(device)

cuda


In [3]:
df_page_test= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/proj/Data/pages/page_test.csv')
df_sent_test= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/proj/Data/sentences/sentence_test.csv')
df_page_test.shape, df_sent_test.shape

((90, 2), (12121, 2))

In [4]:
targetList = ['vector spaces',
 'pos_def_matrices',
 'eigenvec_val',
 'determinants',
 'orthogonality',
 'linear_prog',
 'gauss elim',
 'computations']

In [5]:
df_page_test= pd.concat([df_page_test,pd.get_dummies(df_page_test['label'])],axis=1)

In [6]:
df_page_test.head()

,text,label,computations,determinants,eigenvec_val,gauss elim,linear_prog,orthogonality,pos_def_matrices,vector spaces
0,Chapter 2 Vector Spacesand only a few hundred ...,vector spaces,0,0,0,0,0,0,0,1
1,Chapter 5 Eigenvalues and EigenvectorsProblems...,eigenvec_val,0,0,1,0,0,0,0,0
2,of the columns of Q = I:That is not the only o...,orthogonality,0,0,0,0,0,1,0,0
3,Chapter 2 Vector SpacesYou must notice that th...,vector spaces,0,0,0,0,0,0,0,1
4,5.2 Diagonalization of a MatrixIf A is inverti...,eigenvec_val,0,0,1,0,0,0,0,0


In [7]:
df_sent_test= pd.concat([df_sent_test,pd.get_dummies(df_sent_test['label'])],axis=1)
df_sent_test.head()

,text,label,computations,determinants,eigenvec_val,gauss elim,linear_prog,orthogonality,pos_def_matrices,vector spaces
0,This book begins with the central problem of l...,gauss elim,0,0,0,1,0,0,0,0
1,should work and it does: 4 times (x = −1) plus...,gauss elim,0,0,0,1,0,0,0,0
2,If we stay with determinants (which we don’t p...,gauss elim,0,0,0,1,0,0,0,0
3,"formula to compute the other unknown, x:",gauss elim,0,0,0,1,0,0,0,0
4,The idea of elimination is deceptively simple—...,gauss elim,0,0,0,1,0,0,0,0


In [8]:
class myDataset(torch.utils.data.Dataset):
  def __init__(self,df,tokenizer, maxlen):
    self.df= df
    self.tokenizer = tokenizer
    self.maxlen= maxlen
    self.text = self.df['text']
    self.targets = self.df[targetList].values
  def __len__(self):
    return len(self.text)
  def __getitem__(self,index):
    text= str(self.text[index])
    text= " ".join(text.split())  
    inputs = self.tokenizer.encode_plus(
        text,
        None,
        add_special_tokens= True,
        max_length = self.maxlen,
        padding= 'max_length',
        return_token_type_ids = True,
        truncation = True,
        return_attention_mask = True,
        return_tensors= 'pt'
    )
    return {
            'input_ids':inputs['input_ids'].flatten(),
            'attention_mask':inputs['attention_mask'].flatten(),
            'token_type_ids':inputs['token_type_ids'].flatten(),
            'targets':torch.FloatTensor(self.targets[index])
        }

In [ ]:
pip install transformers

In [10]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
class BERTModel(nn.Module):
    def __init__(self,dr):
        super(BERTModel,self).__init__()
        self.bert= BertModel.from_pretrained("bert-base-uncased", return_dict= True)
        self.dropout = nn.Dropout(dr) 
        self.linear =nn.Linear(768,8)
    def forward(self, input_ids,attention_mask,token_type_ids):
        output = self.bert(input_ids,attention_mask,token_type_ids)
        outputDropout= self.dropout(output.pooler_output)
        output= self.linear(outputDropout)
        return output
page_model = BERTModel(0.3)
sent_model = BERTModel(0.6)
page_model.to(device)
sent_model.to(device)

In [13]:
def load_ckp(ckpt_path,model,optimizer):
    checkpoint= torch.load(ckpt_path, map_location= device)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min= checkpoint['valid_loss_min']
    return model, optimizer, checkpoint['epoch'],valid_loss_min

In [14]:
BATCH_SIZE= 16
MAX_LEN= 256
LR= 1e-4

In [15]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs,targets)
def accuracy(output,targets):
  final_output = torch.sigmoid(output).cpu().detach().numpy()
  return np.sum( np.argmax(targets.cpu().numpy(),axis=1) == np.argmax(final_output, axis=1) ) / BATCH_SIZE *100
  
optimizer1= torch.optim.Adam(params=page_model.parameters(),lr=LR)
optimizer2= torch.optim.Adam(params=sent_model.parameters(),lr=LR)

In [24]:
def test(model, loss_fn, testLoader ):
  test_accList =[]
  test_loss=0
  model.eval() 
  with torch.no_grad():
   for index, batch in enumerate(testLoader):
     input_ids= batch['input_ids'].to(device, dtype= torch.long)
     attention_mask = batch['attention_mask'].to(device, dtype= torch.long)
     token_type_ids= batch['token_type_ids'].to(device, dtype= torch.long)
     targets = batch['targets'].to(device, dtype= torch.float)

     output =model(input_ids,attention_mask,token_type_ids)

     loss= loss_fn(output,targets)
     test_acc= accuracy(output,targets)
     test_accList+= [test_acc]
     test_loss = test_loss + ((1 / (index + 1)) * (loss.item() - test_loss))
     print("TEST BATCH {}/{}====>test loss[{:.8f}] | test accuracy[{:.8f}]".format(index+1,len(testLoader) , test_loss, test_acc ))
     #final_output = torch.sigmoid(output).cpu().detach().numpy()
     #print(np.argmax(targets.cpu().numpy(),axis=1), np.argmax(final_output, axis=1) )
  print("AVG Accuracy = ", sum(test_accList)/len(test_accList))
  return test_accList

In [25]:
def testloader(df):
  testset= myDataset(df,tokenizer,MAX_LEN)
  testLoader= torch.utils.data.DataLoader(
      testset,
      shuffle=False,
      batch_size= BATCH_SIZE,
      num_workers= 0
  )
  return testset, testLoader

In [26]:
page_set,pageLoader=testloader(df_page_test)
sent_set, sentLoader= testloader(df_sent_test)

In [27]:
page_ckp='/content/drive/MyDrive/Colab Notebooks/proj/Data/pages/best.pt'
sent_ckp= '/content/drive/MyDrive/Colab Notebooks/proj/Data/sentences/best.pt'
page_model,optimizer1,e,vlm = load_ckp(page_ckp,page_model,optimizer1)
print(vlm)
sent_model,optimizer2,e,vlm = load_ckp(sent_ckp,sent_model,optimizer2)
print(vlm)

0.026065316051244736
0.05846651007110874


In [ ]:
page_model.state_dict()

In [ ]:
sent_model.state_dict()

In [29]:
print("PAGE MODEL TEST AGAINST PAGE DATASET")
ppList= test(page_model,loss_fn, pageLoader)

PAGE MODEL TEST AGAINST PAGE DATASET
TEST BATCH 1/6====>test loss[0.07778172] | test accuracy[93.75000000]
TEST BATCH 2/6====>test loss[0.04844927] | test accuracy[100.00000000]
TEST BATCH 3/6====>test loss[0.07528755] | test accuracy[81.25000000]
TEST BATCH 4/6====>test loss[0.09192599] | test accuracy[81.25000000]
TEST BATCH 5/6====>test loss[0.09816978] | test accuracy[87.50000000]
TEST BATCH 6/6====>test loss[0.10060729] | test accuracy[56.25000000]
AVG Accuracy =  83.33333333333333


In [30]:
print("SENTENCE MODEL TEST AGAINST SENTENCE DATASET")
ssList =test(sent_model,loss_fn, sentLoader)

SENTENCE MODEL TEST AGAINST SENTENCE DATASET
TEST BATCH 1/758====>test loss[0.17626327] | test accuracy[81.25000000]
TEST BATCH 2/758====>test loss[0.20837490] | test accuracy[75.00000000]
TEST BATCH 3/758====>test loss[0.18197175] | test accuracy[81.25000000]
TEST BATCH 4/758====>test loss[0.18138437] | test accuracy[81.25000000]
TEST BATCH 5/758====>test loss[0.15669663] | test accuracy[93.75000000]
TEST BATCH 6/758====>test loss[0.16370901] | test accuracy[81.25000000]
TEST BATCH 7/758====>test loss[0.17030768] | test accuracy[68.75000000]
TEST BATCH 8/758====>test loss[0.15309876] | test accuracy[93.75000000]
TEST BATCH 9/758====>test loss[0.13683048] | test accuracy[100.00000000]
TEST BATCH 10/758====>test loss[0.12447920] | test accuracy[100.00000000]
TEST BATCH 11/758====>test loss[0.12011157] | test accuracy[93.75000000]
TEST BATCH 12/758====>test loss[0.12753581] | test accuracy[75.00000000]
TEST BATCH 13/758====>test loss[0.12085439] | test accuracy[93.75000000]
TEST BATCH 14

In [31]:
print("SENTENCE MODEL AGAINST PAGE DATASET")
spList = test(sent_model,loss_fn, pageLoader)

SENTENCE MODEL AGAINST PAGE DATASET
TEST BATCH 1/6====>test loss[0.01571073] | test accuracy[100.00000000]
TEST BATCH 2/6====>test loss[0.04851441] | test accuracy[87.50000000]
TEST BATCH 3/6====>test loss[0.08646125] | test accuracy[81.25000000]
TEST BATCH 4/6====>test loss[0.09231676] | test accuracy[87.50000000]
TEST BATCH 5/6====>test loss[0.10564122] | test accuracy[87.50000000]
TEST BATCH 6/6====>test loss[0.10403795] | test accuracy[56.25000000]
AVG Accuracy =  83.33333333333333


In [32]:
print("PAGE MODEL TEST AGAINST SENTENCE DATASET")
psList = test(page_model,loss_fn, sentLoader)

PAGE MODEL TEST AGAINST SENTENCE DATASET
TEST BATCH 1/758====>test loss[0.47705606] | test accuracy[12.50000000]
TEST BATCH 2/758====>test loss[0.52989037] | test accuracy[6.25000000]
TEST BATCH 3/758====>test loss[0.46794006] | test accuracy[43.75000000]
TEST BATCH 4/758====>test loss[0.47315011] | test accuracy[31.25000000]
TEST BATCH 5/758====>test loss[0.43080546] | test accuracy[56.25000000]
TEST BATCH 6/758====>test loss[0.41917715] | test accuracy[56.25000000]
TEST BATCH 7/758====>test loss[0.40166997] | test accuracy[43.75000000]
TEST BATCH 8/758====>test loss[0.39913502] | test accuracy[37.50000000]
TEST BATCH 9/758====>test loss[0.39356350] | test accuracy[43.75000000]
TEST BATCH 10/758====>test loss[0.39310225] | test accuracy[37.50000000]
TEST BATCH 11/758====>test loss[0.39014877] | test accuracy[43.75000000]
TEST BATCH 12/758====>test loss[0.38239400] | test accuracy[56.25000000]
TEST BATCH 13/758====>test loss[0.37596660] | test accuracy[50.00000000]
TEST BATCH 14/758===